In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Entendiendo el negocio

Esta fase inicial se enfoca en la comprensión de los objetivos de proyecto. Después se convierte este conocimiento de los datos en la definición de un problema de minería de datos y en un plan preliminar diseñado para alcanzar los objetivos.

## 2. Comprensión de los datos

En esta fase vamos a realizar un proceso de captura de datos, descripción de los mismos, tareas de exploración y gestión de la calidad de los datos. Para la captura de datos utilizaremos el conjunto de datos del CRM SPERANT el mismo que contiene información inmobiliaria de ventas, el objetivo es poder analizar los datos y explorar la relación que tienen cada una de las variables con la venta realizada. 

In [4]:
## Librerias requeridas
# Para limpieza de datos 
import numpy as np
import pandas as pd

# Para visualización de datos
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [5]:
# Captura de datos de arch
df = pd.read_csv('/kaggle/input/ventasinmobiliaria/CRM SPERANT SQL ALL 202206.csv', delimiter=',', low_memory=False)
df.shape

El Dataframe creado en la captura de datos tiene 280228 filas con 19 atributos. Revisamos esto más a fondo para identificar qué atributos serán necesarios y qué manipulación de datos debe llevarse a cabo antes del análisis exploratorio y el modelado de predicción.

In [6]:
# Revisamos las columnas con valores nulos
df.isnull().sum()

Encontramos que en los 19 atributos existen varias filas con valores nulos

In [7]:
# Obtener valores únicos por cada variable
df.nunique()

In [8]:
# Mostrando los primeros datos como vista previa de la información capturada
df.head()

De la información previa podemos observar datos importantes de clientes como género, estado civil, rango de edad entre otros. No se detallan datos personales puesto que no son necesarios para nuestro análisis. Tenemos también datos de contactos con el cliente como medio de captación, canal de entrada, nivel de interés entre otros, está información puede resultar interesante para encontrar algún patrón en la venta realizada.

Para definir si una venta fue realizada tenemos el atributo fecha_venta, el cual nos servirá para determinar la variable objetivo al momento de entrenar los diferentes modelos.

In [9]:
# Tipos de variables
df.dtypes

La mayoría de las variables son categoricas y solo 4 son continuas

In [10]:
# Se tratan los valores nulos para no perder datos, convirtiendo los nulos en cero
df['cli.total_interacciones'] = np.where(df['cli.total_interacciones'].notna(), df['cli.total_interacciones'], 0)
df['cli.total_interacciones'] = df['cli.total_interacciones'].astype(np.int64)

In [11]:
# Se convierte la variable objetivo fecha_venta en binaria 1 para venta realizada y 0 para no vendidas
# Se convierte la variable fecha_separacion en binaria 1 para fecha separada y 0 para no separada
df['separada'] = np.where(df['uni.fecha_separacion'].notna(), 1, 0)
df['vendido'] = np.where(df['uni.fecha_venta'].notna(), 1, 0)

In [12]:
# Se eliminan columnas ya que fueron convertidas en binarias
df = df.drop(columns=['uni.fecha_separacion', 'uni.fecha_venta'])

In [13]:
# Removiendo valores nulos en el dataframe
df.dropna(inplace = True)

# Creamos un nuevo dataframe para la preparación de los datos
df2 = df


Observamos el nuevo dataframe creado para la preparación de los datos

In [14]:
# Mostramos los datos previos del nuevo dataframe creado para la preparación de los datos
df2.head()

In [15]:
df2.shape

En el análisis exploratorio de los datos vamos a observar de forma visual como se comportan los datos en una primera impresión. Como primera cosa se muestra un gráfico de pastel con la proporción de unidades inmobiliarias vendidas vs no vendidas

In [16]:
labels = 'Vendido', 'No Vendido'
sizes = [df2.vendido[df2['vendido']==1].count(), df2.vendido[df2['vendido']==0].count()]
explode = (0, 0.1)
fig1, ax1 = plt.subplots(figsize=(10, 8))
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("Proporción de unidades inmobiliarias vendidas vs no vendidas", size = 20)
plt.show()

A continuación se muestran gráficos de variables categóricas, en ellos podemos observar la relación que tienen las ventas con cada una de estas variables como género, estado civil, medio captación, rango de edad, estado y último vendedor, 

In [17]:
fig, axarr = plt.subplots(3, 2, figsize=(20, 20))
chart1 = sns.countplot(x='cli.genero', hue = 'vendido',data = df2, ax=axarr[0][0])
chart2 = sns.countplot(x='cli.estado_civil', hue = 'vendido',data = df2, ax=axarr[0][1])
chart3 = sns.countplot(x='cli.medio_captacion', hue = 'vendido',data = df2, ax=axarr[1][0])
chart4 = sns.countplot(x='cli.rango_edad', hue = 'vendido',data = df2, ax=axarr[1][1])
chart5 = sns.countplot(x='cli.estado', hue = 'vendido',data = df2, ax=axarr[2][0])
chart6 = sns.countplot(x='cli.ultimo_vendedor', hue = 'vendido',data = df2, ax=axarr[2][1])

chart1.set_xticklabels(chart1.get_xticklabels(), rotation=40, ha="right")
chart2.set_xticklabels(chart2.get_xticklabels(), rotation=40, ha="right")
chart3.set_xticklabels(chart3.get_xticklabels(), rotation=40, ha="right")
chart4.set_xticklabels(chart4.get_xticklabels(), rotation=40, ha="right")
chart5.set_xticklabels(chart5.get_xticklabels(), rotation=40, ha="right")
chart6.set_xticklabels(chart6.get_xticklabels(), rotation=40, ha="right")

plt.tight_layout()
plt.show()

Mostramos tambíén gráficos de caja para nuestras variables contínuas, total interacciones, area total, separada y precio de venta.

In [18]:
# Relations based on the continuous data attributes
fig, axarr = plt.subplots(2, 2, figsize=(20, 20))
sns.boxplot(y='cli.total_interacciones', x = 'vendido', hue = 'vendido', data = df2, ax=axarr[0][0])
sns.boxplot(y='uni.area_total', x = 'vendido', hue = 'vendido', data = df2, ax=axarr[0][1])
sns.boxplot(y='separada', x = 'vendido', hue = 'vendido', data = df2, ax=axarr[1][0])
sns.boxplot(y='uni.precio_venta', x = 'vendido', hue = 'vendido', data = df2, ax=axarr[1][1])

Se muestra una correlación de variables con nuestra variable objetivo "vendido" y observamos que las variables con una alta relación son total_interacciones, area_total, separada y mínimamente el precio de venta.

In [19]:
# Correlaciones con variables con nuestra variable objetivo vendido
df2.corr()['vendido'].sort_values(ascending = False).head(10).plot(kind='bar')

## 3. Preparación de los datos

Convertimos toas nuestras variables categoricas en variables numércias para esto utilizamos una codificación one hot que es una representación de variables categóricas como vectores binarios. Esto primero requiere que los valores categóricos se asignen a valores enteros.

In [45]:
# Convirtamos todas las variables categóricas en nuevas variables numéricas
df2 = pd.get_dummies(df2)

# Obtenemos nuevamente la correlación de la variable objetivo vendido con las nuevas variables creadas en el proceso anterior
plt.figure(figsize=(15,8))
df2.corr()['vendido'].sort_values(ascending = False).head(10).plot(kind='bar')

La nueva correlación nos permite observar un nuevo comportamiento con valores de variables como por ejemplo, vemos que cuando existe un nivel de interés con minuta firmada existe una mayor probabilidad de venta, seguida por un estado comercial vendido. Así observamos que las nuevas variables creadas en el proceso de One hot encoding nos muestra nuevos patrones de venta.

In [21]:
# Se eliminan columnas que tienen una influencia directa con el resultado optimo del modelo
df2 = df2.drop(columns=['cli.nivel_interes_minuta firmada', 'uni.estado_comercial_vendido'])

## 4. Modelado y Evaluación

### 4.1 Logistic Regression

In [22]:
# Creamos nuevos set de datos para entrenar el modelo
y = df2['vendido'].values
X = df2.drop(columns = ['vendido'])

# Escalando todas las variables al rango de 0 y 1
from sklearn.preprocessing import MinMaxScaler
features = X.columns.values
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X))
X.columns = features

In [23]:
# Creamos el dataset de Entrenamoento y Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [24]:
# Logistic regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
result = model.fit(X_train, y_train)

In [25]:
from sklearn import metrics
prediction_test = model.predict(X_test)

# Mostramos la precisión de la predicción para el modelo
print (metrics.accuracy_score(y_test, prediction_test))

In [26]:
from sklearn.metrics import classification_report

# Reporte de clasificación
print(classification_report(y_test, prediction_test))

In [27]:
# obtener los pesos de todas las variables
weights = pd.Series(model.coef_[0], index=X.columns.values)
print (weights.sort_values(ascending = False)[:10].plot(kind='bar'))

**Observaciones:**

La presición para el modelo de regresión logistica es de 0.98 lo cual es muy bueno para nuestro entrenamiento del modelo, como resultado observamos el peso que tienen las variables de vendedor, estado y precio de venta ya que son determinantes para determinar si se realizó una venta.

### 4.2 Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
model_rf = RandomForestClassifier(n_estimators=1000 , oob_score = True, n_jobs = -1,
                                  random_state =50, max_features = "auto",
                                  max_leaf_nodes = 30)
model_rf.fit(X_train, y_train)

In [29]:
# Make predictions
prediction_test = model_rf.predict(X_test)
print (metrics.accuracy_score(y_test, prediction_test))

In [30]:
print(classification_report(y_test, prediction_test))

In [31]:
importances = model_rf.feature_importances_
weights = pd.Series(importances, index=X.columns.values)
weights.sort_values()[-10:].plot(kind = 'barh')

**Observaciones:**

La presición para el modelo de random forest es de 1.00 lo cual es muy bueno para nuestro entrenamiento del modelo, como resultado observamos el peso que tienen las variables de estado comercial y nivel de interés ya que son determinantes para determinar si se realizó una venta.

### 4.3 Support Vecor Machine (SVM)

In [32]:
# Conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [33]:
from sklearn.svm import SVC

model.svm = SVC(kernel='linear') 
model.svm.fit(X_train,y_train)
preds = model.svm.predict(X_test)
metrics.accuracy_score(y_test, preds)

In [34]:
# Create the Confusion matrix
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,preds)) 

In [35]:
print(classification_report(y_test, preds))

### 4.4 ADA Boost

In [36]:
# AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()

model.fit(X_train,y_train)
preds = model.predict(X_test)
metrics.accuracy_score(y_test, preds)

In [37]:
print(classification_report(y_test, preds))

### 4.5 XG Boost

In [38]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
metrics.accuracy_score(y_test, preds)

In [39]:
print(classification_report(y_test, preds))

### 4.6 Clasification tree

In [41]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

# Conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

DecisionTreeClassifier
DT_MODEL = DecisionTreeClassifier(criterion='entropy', max_depth=4)
DT_MODEL.fit(X_train, y_train)

preds = DT_MODEL.predict(X_test)
metrics.accuracy_score(y_test, preds)

In [48]:
fig, ax = plt.subplots(figsize=(20,20), facecolor='w')
tree.plot_tree(DT_MODEL, feature_names=np.array(X.columns), class_names=['0', '1'], filled=True, ax=ax)

## 5 Conclusión

Nuestro conjunto de datos importados nos da información precisa sobre las ventas realizdas, se lo pudo observar de forma visual en la correlación de variables con las variables de estado comercial y nivel de interés principalmente. Estas dos variables son determinantes para establecer la precisión del modelo ya que observamos que en todos los modelos tenemos una precisión bastante alta al momento de realizar la evaluación de los modelos.

En el modelo de Random Forest observamos algo interesante, además de estas dos variables existen cuatro variables que ayudan a determinar si la venta fue realizada como son precio m2, total interacciones, el precio de venta y el estado.

La variable separada nos dice que el cliente hizo la separación para la compra del inmueble pero no necesariamente termina en compra, ya que tanto en nuestro análisis exploratorio como en las correlaciones y modelos esta variable no juega un papel determinante a la hora de definir una venta realizada.